In [1]:
import os
import sys
sys.path.append('../python_packages_static')
import numpy as np
import rasterio
from rasterio.plot import show
from rasterio.merge import merge
import matplotlib.pyplot as plt
from pyproj import CRS
import flopy
import shutil

# Merge `dis` and `hds` data into single raster coverage from the two parent models that overlap the Neversink domain:
- 0108_0110_0202_0203_MF6_SS_Unconfined_250
- 0204_0206_0209_MF6_SS_Unconfined_250

### Note this will not run correctly in this repo because the large parent model files are not trackted.
Parent model files (by Zell and Sanford, 2020) are available here: https://water.usgs.gov/GIS/metadata/usgswrd/XML/zell2020_wrr.xml

### Merged parent output generated by this notebook is tracked in this repo here: `processed_data\merged_parent_output\parent_mf_nwt`

In [2]:
data_root = '../parent_model/mfexport_outfiles'

parent_models = [
    '0108_0110_0202_0203_MF6_SS_Unconfined_250',
    '0204_0206_0209_MF6_SS_Unconfined_250'
]

packages = os.listdir(os.path.join(data_root, parent_models[0]))

FileNotFoundError: [WinError 3] The system cannot find the path specified: '../parent_model/mfexport_outfiles\\0108_0110_0202_0203_MF6_SS_Unconfined_250'

In [3]:
out_path = '../processed_data/merged_parent_output'

if not os.path.exists(out_path):
    os.makedirs(out_path)

In [4]:
def model_raster_data_merge(data_root, out_path, parent_models, packages):    
    #  get list of tif files from the models to merge
    for pkg in packages:    

        print('Now merging {} package data'.format(pkg))
        package_tifs = []

        for root, dirs, files in os.walk(os.path.join(data_root, parent_models[0], pkg)):
            for file in files:
                if file.endswith('.tif'):
                    package_tifs.append(file)

        for tif in package_tifs:
            src_files_to_merge = []
            for model in parent_models:
                file_path = os.path.join(data_root, model, pkg, 'rasters', tif)
                
                #  open file in read mode w/rasterio and make a list of files to merge
                src = rasterio.open(file_path)
                src_files_to_merge.append(src)

            #  define nondata value by layer
            if tif.startswith('idomain'):
                nodata = 0
            elif tif.startswith(('hds', 'wt')):
                nodata = np.nan
            else:
                nodata = -2.14748365e+09
            print('nodata value: {}'.format(nodata))
            
            #  perform merge with rasterio.merge.merge
            mosaic, out_trans = merge(src_files_to_merge, nodata=nodata)
            print('merged: {}'.format(tif))
            
            #  plot merged layer to make sure that it looks right
            plot_mosaic = mosaic.copy()
            if not tif.startswith('idomain'):
                plot_mosaic[plot_mosaic==nodata] = np.nan
            plt.figure(figsize=(6, 6))
            plt.imshow(plot_mosaic[0])
            plt.colorbar()
            #plt.show()
            
            #  update the metadata with new dimensions, transform and CRS. Also add compression 'lzw'
            out_meta = src.meta.copy()
            out_meta.update({'driver': 'GTiff',
                     'height': mosaic.shape[1],
                     'width': mosaic.shape[2],
                     'transform': out_trans,
                     'compress': 'lzw',  # see if this works or not...
                     'crs': '+datum=WGS84 +lat_0=23 +lat_1=29.5 +lat_2=45.5 +lon_0=-96 +no_defs +proj=aea +units=m +x_0=0 +y_0=0'
                     }
                    )

            #  write mosaic to out_path
            out_fp = os.path.join(out_path, (tif[:-4] + '_merged.tif'))

            with rasterio.open(out_fp, "w", **out_meta) as dest:
                dest.write(mosaic)
            
            print('wrote file: {}'.format(out_fp))
            plt.show()

In [5]:
model_raster_data_merge(data_root, out_path, parent_models, packages)  

NameError: name 'packages' is not defined

### Convert the merged head file to binary format using `flopy.utils`

In [8]:
model_ws = '../processed_data/merged_parent_output/parent_mf_nwt'
'''
if os.path.exists(model_ws):
    shutil.rmtree(model_ws)
    
os.mkdir(model_ws)
'''    
precision = 'double' # or 'double'
dtype = np.float64 # or np.float64

mf = flopy.modflow.Modflow(modelname='ngwm_parent',model_ws=model_ws)
dis = flopy.modflow.ModflowDis(mf, nlay=nlay, nrow=nrow, ncol=ncol, delr=delr, delc=delc)

NameError: name 'nlay' is not defined

In [7]:
hds_tif = 'hds_lay0_per0_stp0_merged.tif'
hds_dir = '../processed_data/merged_parent_output'
hds_path = os.path.join(hds_dir, hds_tif)

with rasterio.open(hds_path) as src:
    data = np.squeeze(src.read())
    
    
plt.imshow(data)
plt.show()

RasterioIOError: ../processed_data/merged_parent_output\hds_lay0_per0_stp0_merged.tif: No such file or directory

replace `nan` nodata with -9999 values

In [ ]:
data[np.isnan(data)] = -9999
plt.imshow(data)
plt.colorbar()
plt.show()

In [ ]:
text = 'head'

# write a binary data file
pertim = dtype(1.0)
header = flopy.utils.BinaryHeader.create(bintype=text, precision=precision,
                                         text=text, nrow=nrow, ncol=ncol,
                                         ilay=1, pertim=pertim,
                                         totim=pertim, kstp=1, kper=1)
pth = os.path.join(model_ws, 'ngwm_parent.hds')
flopy.utils.Util2d.write_bin(data.shape, pth, data, header_data=header)

In [ ]:
mf.write_input()

### write a fake CBC file to make mfsetup happy 

In [ ]:
with open(pth.replace('hds','cbc'), 'w') as ofp:
    ofp.write('This is not the file you are looking for \n')

In [ ]:
hdobj = flopy.utils.binaryfile.HeadFile(pth)

In [ ]:
all_kstpkper = hdobj.get_kstpkper()

In [ ]:
last_steps = {kper: kstp for kstp, kper in all_kstpkper}

In [ ]:
last_steps

In [ ]:
mf.dis.nstp.array

In [ ]:
mf.dis.nper

In [ ]:
# for inset_per, parent_per in self.inset_parent_period_mapping.items():

### update `nam` file with spatial reference information

establish relevant information for parent `.nam` file spatial reference header

In [ ]:
merge_dir = '../parent_model/merged_parent_output/'
file_name = 'idomain_lay0_merged.tif'
input_file = os.path.join(merge_dir, file_name)

In [ ]:
with rasterio.open(input_file) as src:
    meta = src.meta
    bounds = src.bounds

In [ ]:
ncol = meta['width']
nlay = meta['count']
nrow = meta['height']
delc = meta['transform'][0]
delr = -meta['transform'][4]
length_units = 'meters'

# parent grid?
xoff = bounds[0]
yoff = bounds[1]
#  coord ref doesn't match epsg code
proj_string = CRS(meta['crs']).to_proj4()
wkt_string = CRS(meta['crs']).to_wkt()

print('ncol: {0}, nlay: {1}, nrow: {2}, delc: {3}, delr: {4}, length_units: {5}, xoff: {6}, yoff: {7}'.format(ncol, nlay, nrow, delc, delr, length_units, xoff, yoff))

### re-write name file header to include spatial reference information

In [ ]:
nam= [i.strip() for i in open(os.path.join(model_ws,'ngwm_parent.nam'), 'r').readlines() if '#' not in i]
print (nam)

In [ ]:
headerline = ''.join(['#parent nam file just for reading in hds info\n',
      '#xll:{0}; yll:{1}; rotation:0; proj4_str:{2}; units:meters; lenuni:2;  ;start_datetime:1-1-2011'.format(
      xoff, yoff, '+init=epsg:5070')])

In [ ]:
headerline

In [ ]:
with open(os.path.join(model_ws,'ngwm_parent.nam'), 'w') as ofp:
    ofp.write('{}\n'.format(headerline))
    [ofp.write('{}\n'.format(line)) for line in nam]